In [ ]:
# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes
  

In [ ]:
# start streaming video from webcam
from google.colab.patches import cv2_imshow
import cv2
video_stream()
# label for video
label_html = 'Capturing...'
# initialze bounding box to empty
bbox = ''
count = 0
while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break

    # convert JS response to OpenCV Image
    frame = js_to_image(js_reply["img"])
    # create transparent overlay for bounding box
    bbox_array = np.zeros([416,416,4], dtype=np.uint8)
    img=cv2.resize(frame, (416,416))
    results = model(img)
    Label,Bbox,Confidence=[],[],[]
    bbox_array = np.zeros([416,416,4], dtype=np.uint8)
    for res in results.pandas().xyxy:
        # print(len(res))
        for obj in range(len(res)):
            if res['confidence'][obj]>0.0001:
                (x1, y1, x2, y2) = (res['xmin'][obj],res['ymin'][obj],res['xmax'][obj],res['ymax'][obj])
                bbox=(int(x1),int(y1),int(x2),int(y2))
                className = classes[res['class'][obj]]
                Label.append(className)
                Bbox.append(bbox)
                conf=res['confidence'][obj]
                Confidence.append(res['confidence'][obj])
                cv2.rectangle(img, (bbox[0],bbox[1]), (bbox[2],bbox[3]), (36,255,12), 1)
                bbox_array = cv2.rectangle(bbox_array, (bbox[0],bbox[1]), (bbox[2],bbox[3]), (36,255,12), 1)
                text = "{} [{:.2f}]".format(className, float(conf))
                bbox_array = cv2.putText(bbox_array, text, (bbox[0], bbox[1] - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (36, 255, 12), 2)
                break

    bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
    # convert overlay of bbox into bytes
    bbox_bytes = bbox_to_bytes(bbox_array)
    # update bbox so next frame gets new overlay
    bbox = bbox_bytes
    c = cv2.waitKey(1) & 0x100
    if c == ord('q'):  # 'q' key is pressed
        break  # Exit the loop if 'q' is pressed